# Disaster Tweets - LSTM with Embedding Comparison
## Word2Vec vs GloVe vs Random Embedding 성능 비교

**목표:** 80% 이상 정확도 달성 (TF-IDF와 비교 가능한 수준)

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import copy
import os

# NLTK 데이터 다운로드 확인
try:
    nltk.data.find("tokenizers/punkt")
    nltk.data.find("corpora/stopwords")
except:
    nltk.download("punkt")
    nltk.download("stopwords")

# GPU 설정
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

# ========================================
# 1. 데이터 로딩
# ========================================
train_path = '/Users/ys/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/kaggle-projects/disaster-tweets/data/train.csv'
test_path = '/Users/ys/Library/Mobile Documents/com~apple~CloudDocs/Study/AI/Kaggle/kaggle-projects/disaster-tweets/data/test.csv'
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
print(f"Train 데이터: {len(train_df)}개, Test 데이터: {len(test_df)}개")

# ========================================
# 2. 텍스트 전처리 함수
# ========================================
def preprocess_text(df, keyword_mode=None):
    """텍스트 전처리: 소문자 변환, 특수문자 제거, 토큰화, 불용어 제거"""
    df = df.copy()
    if keyword_mode is None:
        keyword_mode = df['keyword'].mode()[0]
    df['keyword'] = df['keyword'].fillna(keyword_mode)
    df['text'] = df['text'].str.lower()
    df['text'] = df['text'].str.replace(r'[^a-z ]', '', regex=True)
    df['tokenized_text'] = df['text'].apply(word_tokenize)
    stop_words = set(stopwords.words('english'))
    df['tokens'] = df['tokenized_text'].apply(
        lambda tokens: [word for word in tokens if word not in stop_words]
    )
    return df, keyword_mode

train_df, keyword_mode = preprocess_text(train_df)
test_df, _ = preprocess_text(test_df, keyword_mode=keyword_mode)
print("전처리 완료!")

# ========================================
# 3. Word2Vec 학습 및 단어 사전 생성
# ========================================
sentences = train_df["tokens"].tolist()
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)
word_to_index = {word: idx + 1 for idx, word in enumerate(w2v_model.wv.index_to_key)}  # 0은 패딩용
vocab_size = len(word_to_index) + 1
print(f"단어 사전 크기: {vocab_size}개")

# ========================================
# 4. 정수 인코딩 및 패딩
# ========================================
def encode_sequences(df, word_to_index, max_len=None):
    """토큰을 정수로 인코딩하고 패딩"""
    encoded = df["tokens"].apply(
        lambda tokens: [word_to_index[t] for t in tokens if t in word_to_index]
    )
    if max_len is None:
        max_len = max(len(s) for s in encoded)
    padded = [seq[:max_len] + [0] * max(0, max_len - len(seq)) for seq in encoded]
    return torch.LongTensor(padded), max_len

X, max_len = encode_sequences(train_df, word_to_index)
y = torch.LongTensor(train_df["target"].values)
X_test, _ = encode_sequences(test_df, word_to_index, max_len)
print(f"시퀀스 최대 길이: {max_len}")

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Train: {len(X_train)}, Val: {len(X_val)}")

# ========================================
# 5. 임베딩 매트릭스 생성 함수들
# ========================================
def create_word2vec_embedding(w2v_model, word_to_index, embedding_dim=100):
    """Word2Vec 임베딩 매트릭스 생성"""
    vocab_size = len(word_to_index) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, idx in word_to_index.items():
        if word in w2v_model.wv:
            embedding_matrix[idx] = w2v_model.wv[word]
    return torch.FloatTensor(embedding_matrix)

def load_glove_embeddings(glove_path, word_to_index, embedding_dim=100):
    """GloVe 임베딩 매트릭스 로드"""
    vocab_size = len(word_to_index) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    embeddings_index = {}
    
    with open(glove_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    
    found = 0
    for word, idx in word_to_index.items():
        if word in embeddings_index:
            embedding_matrix[idx] = embeddings_index[word]
            found += 1
    
    print(f"GloVe: {found}/{len(word_to_index)} 단어 매칭 ({found/len(word_to_index)*100:.1f}%)")
    return torch.FloatTensor(embedding_matrix)

def create_random_embedding(vocab_size, embedding_dim=100):
    """랜덤 임베딩 매트릭스 생성"""
    embedding_matrix = np.random.uniform(-0.25, 0.25, (vocab_size, embedding_dim))
    embedding_matrix[0] = 0  # 패딩은 0
    return torch.FloatTensor(embedding_matrix)

# ========================================
# 6. 개선된 LSTM 모델 (Dropout 추가)
# ========================================
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 num_layers=2, dropout=0.5, pretrained_weights=None, freeze_embedding=False):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        if pretrained_weights is not None:
            self.embedding.weight.data.copy_(pretrained_weights)
            self.embedding.weight.requires_grad = not freeze_embedding
        
        self.lstm = nn.LSTM(
            embedding_dim, 
            hidden_dim, 
            num_layers=num_layers, 
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # *2 for bidirectional
        
    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        output, (hidden, cell) = self.lstm(embedded)
        # Bidirectional: 마지막 forward + backward hidden state 결합
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        hidden = self.dropout(hidden)
        return self.fc(hidden)

# ========================================
# 7. Dataset 및 DataLoader
# ========================================
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ========================================
# 8. Early Stopping 클래스
# ========================================
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.best_model = None
        self.early_stop = False
        
    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            self.counter = 0
        return self.early_stop

# ========================================
# 9. 학습 및 평가 함수
# ========================================
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss, total_correct, total_samples = 0, 0, 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(1), labels.float())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()
        total_loss += loss.item()
        preds = torch.round(torch.sigmoid(outputs))
        total_correct += (preds.squeeze(1) == labels).sum().item()
        total_samples += labels.size(0)
    return total_loss / len(loader), total_correct / total_samples

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct, total_samples = 0, 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(1), labels.float())
            total_loss += loss.item()
            preds = torch.round(torch.sigmoid(outputs))
            total_correct += (preds.squeeze(1) == labels).sum().item()
            total_samples += labels.size(0)
    return total_loss / len(loader), total_correct / total_samples

# ========================================
# 10. 전체 학습 함수
# ========================================
def train_model(embedding_matrix, embedding_name, epochs=25, batch_size=32, lr=0.001):
    print(f"\n{'='*50}")
    print(f"Training with {embedding_name} Embedding")
    print(f"{'='*50}")
    
    # DataLoader 생성
    train_dataset = TextDataset(X_train, y_train)
    val_dataset = TextDataset(X_val, y_val)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # 모델 생성
    model = LSTMClassifier(
        vocab_size=vocab_size,
        embedding_dim=100,
        hidden_dim=128,
        output_dim=1,
        num_layers=2,
        dropout=0.5,
        pretrained_weights=embedding_matrix
    ).to(device)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)
    early_stopping = EarlyStopping(patience=5)
    
    best_val_acc = 0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    for epoch in range(epochs):
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        scheduler.step(val_loss)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
        
        print(f'Epoch {epoch+1:02}/{epochs} | '
              f'Train Loss: {train_loss:.4f}, Acc: {train_acc*100:.2f}% | '
              f'Val Loss: {val_loss:.4f}, Acc: {val_acc*100:.2f}%')
        
        if early_stopping(val_loss, model):
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    # 최적 모델 복원
    model.load_state_dict(early_stopping.best_model)
    
    return model, best_val_acc, history

# ========================================
# 11. 임베딩 비교 실험
# ========================================
results = {}

# 1) Word2Vec 임베딩
w2v_embedding = create_word2vec_embedding(w2v_model, word_to_index)
w2v_model_trained, w2v_acc, w2v_history = train_model(w2v_embedding, "Word2Vec")
results['Word2Vec'] = w2v_acc

# 2) GloVe 임베딩 (파일 경로 설정 필요)
glove_path = '/Users/ys/Downloads/study/models/glove/glove.6B.100d.txt'
if os.path.exists(glove_path):
    glove_embedding = load_glove_embeddings(glove_path, word_to_index)
    glove_model_trained, glove_acc, glove_history = train_model(glove_embedding, "GloVe")
    results['GloVe'] = glove_acc
else:
    print(f"\n⚠️ GloVe 파일을 찾을 수 없습니다: {glove_path}")
    print("GloVe 다운로드: https://nlp.stanford.edu/data/glove.6B.zip")
    results['GloVe'] = None

# 3) Random 임베딩 (Baseline)
random_embedding = create_random_embedding(vocab_size)
random_model_trained, random_acc, random_history = train_model(random_embedding, "Random (Baseline)")
results['Random'] = random_acc

# ========================================
# 12. 결과 비교 표
# ========================================
print("\n" + "="*60)
print("📊 임베딩 비교 실험 결과")
print("="*60)
print(f"{'Embedding':<15} | {'Val Accuracy':<15} | {'비고'}")
print("-"*60)
for name, acc in results.items():
    if acc is not None:
        note = "✅ Best" if acc == max(v for v in results.values() if v) else ""
        print(f"{name:<15} | {acc*100:.2f}%{'':<9} | {note}")
    else:
        print(f"{name:<15} | {'N/A':<15} | 파일 없음")
print("="*60)

# 최고 성능 모델 선택
best_embedding = max((k for k, v in results.items() if v), key=lambda k: results[k])
print(f"\n🏆 최고 성능: {best_embedding} ({results[best_embedding]*100:.2f}%)")

# ========================================
# 13. 테스트 데이터 예측 (최고 성능 모델 사용)
# ========================================
# 최고 성능 모델 선택
if best_embedding == 'Word2Vec':
    best_model = w2v_model_trained
elif best_embedding == 'GloVe' and results['GloVe'] is not None:
    best_model = glove_model_trained
else:
    best_model = random_model_trained

class TestDataset(Dataset):
    def __init__(self, X):
        self.X = X
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx]

test_dataset = TestDataset(X_test.to(device))
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

best_model.eval()
test_preds = []
with torch.no_grad():
    for inputs in test_loader:
        outputs = best_model(inputs)
        preds = torch.round(torch.sigmoid(outputs))
        test_preds.extend(preds.cpu().numpy().flatten().astype(int))

# 제출 파일 생성
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'target': test_preds
})
submission_path = 'submission_lstm_enh.csv'
submission_df.to_csv(submission_path, index=False)
print(f"\n✅ '{submission_path}' 파일 생성 완료!")
print(submission_df.head())

Using device: mps
Train 데이터: 7613개, Test 데이터: 3263개
전처리 완료!
단어 사전 크기: 6385개
시퀀스 최대 길이: 20
Train: 6090, Val: 1523

Training with Word2Vec Embedding
Epoch 01/25 | Train Loss: 0.5975, Acc: 66.78% | Val Loss: 0.4558, Acc: 79.84%
Epoch 02/25 | Train Loss: 0.3836, Acc: 83.56% | Val Loss: 0.4457, Acc: 80.89%
Epoch 03/25 | Train Loss: 0.2935, Acc: 88.31% | Val Loss: 0.4835, Acc: 77.81%
Epoch 04/25 | Train Loss: 0.2348, Acc: 90.85% | Val Loss: 0.5612, Acc: 79.25%
Epoch 05/25 | Train Loss: 0.1905, Acc: 92.81% | Val Loss: 0.6232, Acc: 78.59%
Epoch 06/25 | Train Loss: 0.1517, Acc: 94.19% | Val Loss: 0.6714, Acc: 77.61%
Epoch 07/25 | Train Loss: 0.1094, Acc: 96.01% | Val Loss: 0.8403, Acc: 76.95%
Early stopping at epoch 7
GloVe: 5799/6384 단어 매칭 (90.8%)

Training with GloVe Embedding
Epoch 01/25 | Train Loss: 0.5102, Acc: 75.40% | Val Loss: 0.4385, Acc: 80.17%
Epoch 02/25 | Train Loss: 0.4452, Acc: 80.07% | Val Loss: 0.4703, Acc: 80.96%
Epoch 03/25 | Train Loss: 0.4108, Acc: 82.32% | Val Loss: 0.406